In [1]:
!pip install imutils

In [3]:
!wget https://github.com/RufinaMay/CV2019Fall_Pictures/raw/master/deploy.prototxt.txt
!wget https://github.com/RufinaMay/CV2019Fall_Pictures/raw/master/res10_300x300_ssd_iter_140000.caffemodel
# clear_output()

--2020-10-19 03:37:32--  https://github.com/RufinaMay/CV2019Fall_Pictures/raw/master/deploy.prototxt.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/RufinaMay/CV2019Fall_Pictures/master/deploy.prototxt.txt [following]
--2020-10-19 03:37:32--  https://raw.githubusercontent.com/RufinaMay/CV2019Fall_Pictures/master/deploy.prototxt.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28092 (27K) [text/plain]
Saving to: ‘deploy.prototxt.txt.1’

deploy.prototxt.txt 100%[===================>]  27.43K  --.-KB/s    in 0.01s   

2020-10-19 03:37:32 (2.19 MB/s) - ‘deploy.prototxt.txt.1’ saved [28092/28092]



In [15]:
# base path to YOLO directory
MODEL_PATH = "yolo-coco"

MIN_CONF = 0.3
NMS_THRESH = 0.3
USE_GPU = False



from IPython.display import clear_output
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import clear_output
# set inline plots size
plt.rcParams["figure.figsize"] = (16, 10) # (w, h)
import numpy as np
import time
import os
import cv2
import zipfile
import tarfile
from collections import namedtuple
from google.colab.patches import cv2_imshow
from xml.dom import minidom

In [5]:
!wget https://s3-us-west-2.amazonaws.com/static.pyimagesearch.com/opencv-yolo/yolo-object-detection.zip?__s=1essnpgyhz7jwwcpjszi -O yolo-object-detection.zip
# clear_output()

--2020-10-19 03:37:43--  https://s3-us-west-2.amazonaws.com/static.pyimagesearch.com/opencv-yolo/yolo-object-detection.zip?__s=1essnpgyhz7jwwcpjszi
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.205.168
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.205.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308184552 (294M) [application/zip]
Saving to: ‘yolo-object-detection.zip’

yolo-object-detecti 100%[===================>] 293.91M  36.6MB/s    in 8.6s    

2020-10-19 03:37:52 (34.1 MB/s) - ‘yolo-object-detection.zip’ saved [308184552/308184552]



In [6]:
# UNZIP YOLO
with zipfile.ZipFile('yolo-object-detection.zip', 'r') as zip_ref:
    zip_ref.extractall('yolo_data')

In [8]:
import cv2
import numpy as np

scale = 0.00392
classes_file = "/content/yolo_data/yolo-object-detection/yolo-coco/coco.names"
weights = "/content/yolo_data/yolo-object-detection/yolo-coco/yolov3.weights"
config_file = "/content/yolo_data/yolo-object-detection/yolo-coco/yolov3.cfg"
classes = None
LABELS = open(classes_file).read().strip().split("\n")

with open(classes_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# read pre-trained model and config file
net = cv2.dnn.readNet(weights, config_file)
LABELS

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [16]:
# Easy to use car detector that takes an image and ouputs a list of bounding boxes
class PersonDetector():
  def __init__(self):
    module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"
    self.detector = hub.load(module_handle).signatures['default']
  
  def filter_boxes(self, image, boxes, class_names, scores, max_boxes=10, min_score=0.6, class_of_interest='Person'):
    im_height, im_width = image.shape[:2]
    state=[]

    for i in range(min(boxes.shape[0], max_boxes)):
      if scores[i] >= min_score and class_names[i].decode("ascii")==class_of_interest:
        [ymin, xmin, ymax, xmax] = boxes[i]
        (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
        left, right, top, bottom = min(left, right), max(left, right), min(top, bottom), max(top, bottom)
        left, right, top, bottom  = int(left), int(right), int(top), int(bottom) 
        state.append((left, top, right-left, bottom-top))
    return state

  def detect(self, img):
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    result = self.detector(converted_img)

    result = {key:value.numpy() for key,value in result.items()}
    bbox = self.filter_boxes(img, result["detection_boxes"],
        result["detection_class_entities"], result["detection_scores"])
    return bbox

In [17]:
def detect_people(frame, net, ln, personIdx=0):
	# grab the dimensions of the frame and  initialize the list of
	# results
	(H, W) = frame.shape[:2]
	results = []

	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
 
 
	net.setInput(blob)
	layerOutputs = net.forward(ln)

	# initialize our lists of detected bounding boxes, centroids, and
	# confidences, respectively
	boxes = []
	centroids = []
	confidences = []

	# loop over each of the layer outputs
	for output in layerOutputs:
		# loop over each of the detections
		for detection in output:
			# extract the class ID and confidence (i.e., probability)
			# of the current object detection
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			# filter detections by (1) ensuring that the object
			# detected was a person and (2) that the minimum
			# confidence is met
			if classID == personIdx and confidence > MIN_CONF:
				# scale the bounding box coordinates back relative to
				# the size of the image, keeping in mind that YOLO
				# actually returns the center (x, y)-coordinates of
				# the bounding box followed by the boxes' width and
				# height
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				# use the center (x, y)-coordinates to derive the top
				# and and left corner of the bounding box
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				# update our list of bounding box coordinates,
				# centroids, and confidences
				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

	# apply non-maxima suppression to suppress weak, overlapping
	# bounding boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

	# ensure at least one detection exists
	if len(idxs) > 0:
		# loop over the indexes we are keeping
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			# update our results list to consist of the person
			# prediction probability, bounding box coordinates,
			# and the centroid
			r = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(r)
	return results

In [18]:
 def read_and_resize_image(filename, grayscale = False, fx= 1, fy=1):
    if grayscale:
      img_result = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
      imgbgr = cv2.imread(filename, cv2.IMREAD_COLOR)
      img_result = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2RGB)
    img_result = cv2.resize(img_result, None, fx=fx, fy=fy, interpolation = cv2.INTER_CUBIC)
    return img_result
def showInRow(list_of_images, titles = None, disable_ticks = False):
  count = len(list_of_images)
  for idx in range(count):
    subplot = plt.subplot(1, count, idx+1)
    if titles is not None:
      subplot.set_title(titles[idx])
      
    img = list_of_images[idx]
    cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
    subplot.imshow(img, cmap=cmap)
    if disable_ticks:
      plt.xticks([]), plt.yticks([])
  plt.show()

In [19]:
from scipy.spatial import distance as dist
import imutils
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
ln


['yolo_82', 'yolo_94', 'yolo_106']

In [27]:
!wget "https://drive.google.com/uc?export=view&id=1rHIUOstB87dtHDWgzbWhg5hmhqo-BHik" -O pedestrians.mp4
clear_output()

In [ ]:
# def process_video(video_path,out, num_frames=30, start_frame=0):]
MIN_DISTANCE = 50
video_path = "/content/walk.mp4"
vid = cv2.VideoCapture(video_path)
# vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
writer = None
# try:
while True:
  ret, frame = vid.read()
  if not ret:
    vid.release()
    break
  frame = imutils.resize(frame, width=700)
  # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  results = detect_people(frame, net, ln, personIdx=LABELS.index("person"))
  violate = set()

  if len(results) >= 2:
    centroids = np.array([r[2] for r in results])
    D = dist.cdist(centroids, centroids, metric="euclidean")
    # loop over the upper triangular of the distance matrix
    for i in range(0, D.shape[0]):
      for j in range(i + 1, D.shape[1]):
        # check to see if the distance between any two
        # centroid pairs is less than the configured number
        # of pixels
        if D[i, j] < MIN_DISTANCE:
          # update our violation set with the indexes of
          # the centroid pairs
          violate.add(i)
          violate.add(j)

  for (i, (prob, bbox, centroid)) in enumerate(results):
    (startX, startY, endX, endY) = bbox
    (cX, cY) = centroid
    color = (0, 255, 0)
    # colorRed = (255,0, 0)
    if i in violate:
      color = (0, 0, 255)

    cv2.circle(frame, (cX, cY), int((endY-startY)/2),color, 2)
    cv2.circle(frame, (cX, cY), 5, color, 2)

  text = "Violations: {}".format(len(violate))
  cv2.putText(frame, text, (10, frame.shape[0] - 25),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  # showInRow([frame])
  if writer is  None:
    writer =cv2.VideoWriter('dista_detector.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame.shape[1], frame.shape[0]),True)
  # out.write(frame)
  if writer is not None:
      writer.write(frame)
# except KeyboardInterrupt:
  # vid.release()


